In [1]:
import sqlite3
from textwrap import dedent
from typing import List
import sqlite_vec
from fastembed import TextEmbedding
from groq import Groq
from groq.types.chat import ChatCompletionMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sqlite_vec import serialize_float32
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = Groq(api_key = os.environ["GROQ_API_KEY"])
MODEL = "llama-3.1-70b-versatile"
TEMPERATURE = 0

In [3]:
dbfile = 'data/readmes.sqlite3'
# Create a SQL connection to our SQLite database
db = sqlite3.connect(dbfile)
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [4]:
embedding_model = TextEmbedding()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
def call_model(prompt: str, messages=[]) -> ChatCompletionMessage:
    messages.append(
        {
            "role": "user",
            "content": prompt,
        }
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
    )
    return response.choices[0].message.content

In [6]:
query = "¿A quíen mató José Arcadio Buendía?"
query_embeddings = list(embedding_model.embed([query]))
query_embedding = query_embeddings[0]

In [7]:
results = db.execute(
    """
      SELECT
        chunk_embeddings.id,
        distance,
        text
      FROM chunk_embeddings
      LEFT JOIN chunks ON chunks.id = chunk_embeddings.id
      WHERE embedding MATCH ?
        AND k = 3
      ORDER BY distance
    """,
    [serialize_float32(query_embedding)],
).fetchall()

In [8]:
results

[(2,
  0.6349784135818481,
  'En Macondo, José Arcadio Buendía alternaba entre sus intentos de construir la ciudad de sus sueños y la educación de sus hijos. El primogénito, José Arcadio, se desarrolló rápidamente, mientras que Aureliano mostraba interés por la alquimia. Úrsula quedó embarazada por tercera vez.\nUna mujer llamada Pilar Ternera, que leía el futuro en las cartas, comenzó a visitar la casa. Predijo que José Arcadio sería feliz y tuvo un encuentro sexual con él en un granero. Este episodio despertó en el joven una intensa obsesión sexual y emocional. José Arcadio empezó a buscar a Pilar Ternera todas las noches, desarrollando una relación clandestina que marcaría profundamente su adolescencia y tendría consecuencias significativas en el futuro. La presencia de Pilar Ternera en la vida de los Buendía se convertiría en un elemento recurrente y crucial en la historia familiar.\nMientras tanto, José Arcadio Buendía y Aureliano lograron separar oro en sus experimentos alquímico

In [8]:
def retrieve_context(
    query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model
) -> str:
    query_embedding = list(embedding_model.embed([query]))[0]
    results = db.execute(
        """
    SELECT
        chunk_embeddings.id,
        distance,
        text
    FROM chunk_embeddings
    LEFT JOIN chunks ON chunks.id = chunk_embeddings.id
    WHERE embedding MATCH ? AND k = ?
    ORDER BY distance
        """,
        [serialize_float32(query_embedding), k],
    ).fetchall()
    return "\n-----\n".join([item[2] for item in results])

In [9]:
print(retrieve_context(query))

En Macondo, José Arcadio Buendía alternaba entre sus intentos de construir la ciudad de sus sueños y la educación de sus hijos. El primogénito, José Arcadio, se desarrolló rápidamente, mientras que Aureliano mostraba interés por la alquimia. Úrsula quedó embarazada por tercera vez.
Una mujer llamada Pilar Ternera, que leía el futuro en las cartas, comenzó a visitar la casa. Predijo que José Arcadio sería feliz y tuvo un encuentro sexual con él en un granero. Este episodio despertó en el joven una intensa obsesión sexual y emocional. José Arcadio empezó a buscar a Pilar Ternera todas las noches, desarrollando una relación clandestina que marcaría profundamente su adolescencia y tendría consecuencias significativas en el futuro. La presencia de Pilar Ternera en la vida de los Buendía se convertiría en un elemento recurrente y crucial en la historia familiar.
Mientras tanto, José Arcadio Buendía y Aureliano lograron separar oro en sus experimentos alquímicos. El desinterés de José Arcadio

In [10]:
SYSTEM_PROMPT = """
You're an expert AI/ML engineer with a background in software development. You're
answering questions about technical topics and projects.
If you don't know the answer - just reply with an excuse that you
don't know. Keep your answers brief and to the point. Be kind and respectful.

Use the provided context for your answers. The most relevant information is
at the top. Each piece of information is separated by ---.
"""

In [11]:
def ask_question(query: str) -> str:
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
    ]
    context = retrieve_context(query)
    prompt = dedent(
        f"""
Use the following information:

```
{context}
```

to answer the question:
{query}
    """
    )
    return call_model(prompt, messages), context

In [12]:
query = "¿A quíen veria Úrsula después de consumar su matrimonio?"
response, context = ask_question(query)
print(response)

Después de consumar su matrimonio, Úrsula comenzó a ver el fantasma de Prudencio Aguilar.


In [18]:
query = "¿Con quíen se batió en duelo José Arcadio Buendía?"
response, context = ask_question(query)
print(response)

José Arcadio Buendía se batió en duelo con Prudencio Aguilar.


In [17]:
query = "¿A quíen mató el papá Aureliano Buendía?"
response, context = ask_question(query)
print(response)

Según el texto, el papá de Aureliano Buendía, José Arcadio Buendía, mató a Prudencio Aguilar con una lanza después de que este lo insultó sobre su supuesta impotencia.


In [19]:
query = "¿Por qué mató el papá de Amaranta a Prudencio Aguilar?"
response, context = ask_question(query)
print(response)

El papá de Amaranta, José Arcadio Buendía, mató a Prudencio Aguilar porque este último lo insultó sobre su supuesta impotencia.


In [15]:
query = "¿A quíen era la hermana menor de Aureliano Buendía?"
response, context = ask_question(query)
print(response)

Amaranta era la hermana menor de Aureliano Buendía.


In [16]:
query = "¿Con quíen se batió en duelo papá de Aureliano Buendía?"
response, context = ask_question(query)
print(response)

José Arcadio Buendía se batió en duelo con Prudencio Aguilar.
